In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('fred_quarterly.csv')

In [3]:
df.head()

,date,month,quarter,Target,gnp,gdpdef,gdp
0,1/1/2000,2000-01,2000Q1,12935.252,10035.263,77.325,10002.179
1,4/1/2000,2000-04,2000Q2,13170.749,10283.391,77.807,10247.720
2,7/1/2000,2000-07,2000Q3,13183.890,10348.645,78.263,10318.165
3,10/1/2000,2000-10,2000Q4,13262.250,10490.430,78.688,10435.744
4,1/1/2001,2001-01,2001Q1,13219.251,10510.297,79.204,10470.231


In [4]:
df.shape

(86, 7)

In [11]:
from pycaret.regression import *
clf1 = setup(data = df,target = 'Target')

,Description,Value
0,session_id,812
1,Target,Target
2,Original Data,"(86, 7)"
3,Missing Values,True
4,Numeric Features,3
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(60, 18)"


In [12]:
lr = create_model('lr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,102.1883,18722.6152,136.8306,0.9956,0.0101,0.0073
1,89.7407,11785.7217,108.5621,0.9969,0.0077,0.0061
2,128.0080,32605.4668,180.5698,0.9898,0.0109,0.0079
3,166.3761,36525.3633,191.1161,0.9790,0.0124,0.0106
4,123.3260,22768.2402,150.8915,0.9904,0.0105,0.0083
5,73.2612,11239.4834,106.0164,0.9970,0.0070,0.0047
6,1526.1396,12454885.0000,3529.1479,-1.6998,0.1508,0.0799
7,128.5550,19528.5488,139.7446,0.9942,0.0095,0.0086
8,125.5366,24901.3145,157.8015,0.9902,0.0113,0.0087
9,110.2489,16728.9824,129.3406,0.9949,0.0082,0.0070


In [13]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [14]:
rf = create_model('rf', fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,72.4173,10963.9394,104.7088,0.9973,0.0062,0.0043
1,152.7519,43071.9278,207.5378,0.9828,0.0126,0.0093
2,135.5779,27204.8201,164.9388,0.9912,0.0104,0.0085
3,238.8567,317495.7666,563.4676,0.9214,0.0308,0.0132
4,155.1663,30984.9244,176.0254,0.9898,0.0115,0.0099
Mean,150.9540,85944.2756,243.3357,0.9765,0.0143,0.0090
SD,53.2020,116229.9754,163.4993,0.0279,0.0085,0.0029


In [15]:
svm = create_model('svm', cross_validation = False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1377.5554,2.955244e+06,1719.0824,-0.0414,0.1073,0.0855


In [17]:
lgbms = [create_model('lightgbm', learning_rate = i) for i in np.arange(0.1,1,0.1)]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,707.5009,735764.5008,857.7672,0.8273,0.0562,0.0476
1,501.4655,394993.5641,628.4851,0.8945,0.0403,0.0319
2,938.1452,986276.7894,993.1147,0.6920,0.0610,0.0569
3,662.9701,615499.2454,784.5376,0.6460,0.0470,0.0400
4,509.2523,426110.2252,652.7712,0.8208,0.0444,0.0339
5,771.1710,693620.3413,832.8387,0.8119,0.0511,0.0467
6,736.0256,2013531.8850,1418.9897,0.5635,0.0808,0.0397
7,736.3403,658544.0733,811.5073,0.8056,0.0522,0.0474
8,885.4946,1021144.0869,1010.5167,0.5985,0.0641,0.0580
9,529.1301,566204.8627,752.4659,0.8270,0.0449,0.0324


In [19]:
dt = create_model('dt')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,175.6429,62772.4837,250.5444,0.9853,0.0148,0.0105
1,159.7409,39983.8995,199.9597,0.9893,0.0114,0.0096
2,227.9134,66703.9541,258.2711,0.9792,0.0150,0.0133
3,236.3970,79394.2353,281.7698,0.9543,0.0178,0.0146
4,113.5007,25199.6995,158.7441,0.9894,0.0099,0.0070
5,152.6356,30315.5097,174.1135,0.9918,0.0117,0.0097
6,689.4235,1855519.6975,1362.1746,0.5978,0.0773,0.0373
7,142.5837,26827.6079,163.7914,0.9921,0.0107,0.0093
8,142.9632,23734.7532,154.0609,0.9907,0.0105,0.0095
9,267.0724,82275.2325,286.8366,0.9749,0.0174,0.0163


In [20]:
tuned_dt = tune_model(dt)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,273.0731,138076.0155,371.5858,0.9676,0.0215,0.0161
1,217.4334,62373.7926,249.7475,0.9833,0.0159,0.0137
2,253.4919,83234.2961,288.5035,0.9740,0.0173,0.0153
3,210.9162,52875.1920,229.9461,0.9696,0.0147,0.0133
4,240.2021,112234.5669,335.0143,0.9528,0.0196,0.0143
5,257.8088,79225.1576,281.4696,0.9785,0.0184,0.0164
6,833.0021,1598355.6261,1264.2609,0.6535,0.0736,0.0491
7,197.2779,51172.8084,226.2141,0.9849,0.0144,0.0126
8,411.1026,213391.1872,461.9428,0.9161,0.0287,0.0264
9,255.4858,87879.5843,296.4449,0.9732,0.0178,0.0156


In [21]:
tuned_dt = tune_model(dt, n_iter = 50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,157.8948,31283.4559,176.8713,0.9927,0.0108,0.0097
1,138.2807,26900.7602,164.0145,0.9928,0.0103,0.0086
2,181.4508,44854.5626,211.7890,0.9860,0.0122,0.0107
3,248.5248,67548.2221,259.9004,0.9611,0.0162,0.0153
4,184.7073,46218.4985,214.9849,0.9806,0.0127,0.0112
5,184.9115,41328.7340,203.2947,0.9888,0.0138,0.0119
6,743.9011,1672407.1069,1293.2158,0.6375,0.0738,0.0421
7,105.3094,15415.9921,124.1612,0.9954,0.0084,0.0070
8,214.8156,48308.7562,219.7925,0.9810,0.0143,0.0140
9,185.4347,38572.0539,196.3977,0.9882,0.0123,0.0116


In [22]:
tuned_dt = tune_model(dt, optimize = 'MAE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,273.0731,138076.0155,371.5858,0.9676,0.0215,0.0161
1,217.4334,62373.7926,249.7475,0.9833,0.0159,0.0137
2,253.4919,83234.2961,288.5035,0.9740,0.0173,0.0153
3,210.9162,52875.1920,229.9461,0.9696,0.0147,0.0133
4,240.2021,112234.5669,335.0143,0.9528,0.0196,0.0143
5,257.8088,79225.1576,281.4696,0.9785,0.0184,0.0164
6,833.0021,1598355.6261,1264.2609,0.6535,0.0736,0.0491
7,197.2779,51172.8084,226.2141,0.9849,0.0144,0.0126
8,411.1026,213391.1872,461.9428,0.9161,0.0287,0.0264
9,255.4858,87879.5843,296.4449,0.9732,0.0178,0.0156


In [27]:
plot_model(tuned_dt, plot = 'parameter')

,Parameters
ccp_alpha,0.0
criterion,mae
max_depth,7
max_features,1.0
max_leaf_nodes,None
min_impurity_decrease,0.005
min_impurity_split,None
min_samples_leaf,5
min_samples_split,9
min_weight_fraction_leaf,0.0


In [28]:
bagged_dt = ensemble_model(dt)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,53.8515,9717.0669,98.5752,0.9977,0.0063,0.0035
1,78.3320,11396.0306,106.7522,0.9970,0.0062,0.0047
2,166.0216,40417.8081,201.0418,0.9874,0.0115,0.0098
3,209.1351,56468.0664,237.6301,0.9675,0.0151,0.0130
4,51.9625,4640.4404,68.1208,0.9980,0.0044,0.0033
5,119.4232,19038.4409,137.9799,0.9948,0.0086,0.0075
6,444.9124,863292.2392,929.1352,0.8129,0.0511,0.0240
7,128.6610,32796.1799,181.0972,0.9903,0.0109,0.0078
8,146.5211,28571.5446,169.0312,0.9888,0.0116,0.0098
9,138.5234,27595.2902,166.1183,0.9916,0.0105,0.0086


In [29]:
bagged_dt

BaggingRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                      criterion='mse',
                                                      max_depth=None,
                                                      max_features=None,
                                                      max_leaf_nodes=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      presort='deprecated',
                                                      random_state=812,
                                                      splitter='best'),
                 bootstrap=Tr

In [30]:
boosted_dt = ensemble_model(dt, method = 'Boosting')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,94.3358,16343.7928,127.8428,0.9962,0.0078,0.0057
1,131.2944,22740.7700,150.8004,0.9939,0.0088,0.0080
2,167.3807,45292.5191,212.8204,0.9859,0.0120,0.0097
3,209.3123,59150.9566,243.2097,0.9660,0.0152,0.0129
4,119.5394,24035.6992,155.0345,0.9899,0.0097,0.0074
5,127.2142,24803.1255,157.4901,0.9933,0.0093,0.0075
6,287.4590,205295.9292,453.0959,0.9555,0.0247,0.0165
7,157.4518,40887.6661,202.2070,0.9879,0.0128,0.0100
8,187.8193,36652.8244,191.4493,0.9856,0.0127,0.0123
9,150.4144,24558.5973,156.7118,0.9925,0.0099,0.0094


In [31]:
bagged_dt2 = ensemble_model(dt, n_estimators=50)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,82.6846,14710.1047,121.2852,0.9965,0.0076,0.0051
1,52.2097,7101.8811,84.2727,0.9981,0.0046,0.0030
2,146.0052,39999.2746,199.9982,0.9875,0.0112,0.0085
3,201.6993,55783.8253,236.1860,0.9679,0.0147,0.0124
4,61.6746,6318.6668,79.4900,0.9973,0.0052,0.0040
5,82.3705,9018.9104,94.9679,0.9976,0.0058,0.0050
6,416.9855,779677.8810,882.9937,0.8310,0.0484,0.0223
7,104.9697,20970.3935,144.8116,0.9938,0.0089,0.0065
8,157.8931,28464.0069,168.7128,0.9888,0.0113,0.0104
9,143.1096,30132.2873,173.5865,0.9908,0.0108,0.0087


In [35]:
huber = create_model('huber', verbose = False)
dt = create_model('dt', verbose = False)
lightgbm = create_model('lightgbm', verbose = False)
ridge = create_model('ridge', verbose = False)

In [36]:
blend_specific = blend_models(estimator_list = [huber,dt,lightgbm,ridge])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,245.1088,84191.1490,290.1571,0.9802,0.0190,0.0164
1,186.2668,46928.7651,216.6305,0.9875,0.0138,0.0119
2,310.8109,122475.2887,349.9647,0.9617,0.0217,0.0192
3,289.2203,99881.6193,316.0405,0.9425,0.0194,0.0177
4,241.4368,72725.4912,269.6766,0.9694,0.0180,0.0159
5,257.3031,82947.0587,288.0053,0.9775,0.0176,0.0156
6,722.8052,2307101.1497,1518.9145,0.4999,0.0720,0.0384
7,247.5174,69490.1727,263.6099,0.9795,0.0167,0.0159
8,314.3183,130046.8920,360.6201,0.9489,0.0236,0.0208
9,240.4328,67677.2357,260.1485,0.9793,0.0166,0.0153


In [37]:
stack_1 = stack_models([huber,dt,lightgbm,ridge])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,94.3782,14471.9222,120.2993,0.9966,0.0086,0.0066
1,96.7650,12907.5537,113.6114,0.9966,0.0080,0.0066
2,113.0172,27527.5797,165.9144,0.9914,0.0094,0.0067
3,140.0582,30092.8377,173.4729,0.9827,0.0114,0.0091
4,114.4579,17812.2405,133.4625,0.9925,0.0093,0.0077
5,53.7863,4354.5867,65.9893,0.9988,0.0042,0.0034
6,2563.6597,35291953.6855,5940.7031,-6.6500,0.2290,0.1343
7,112.3484,14831.2713,121.7837,0.9956,0.0081,0.0074
8,123.5804,20344.5699,142.6344,0.9920,0.0101,0.0084
9,110.0154,16885.9787,129.9461,0.9948,0.0081,0.0069


In [38]:
stack_1 = stack_models([huber,dt,lightgbm], meta_model = ridge)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,102.1913,12737.9987,112.8627,0.9970,0.0078,0.0068
1,99.6214,12431.6613,111.4974,0.9967,0.0076,0.0066
2,112.3158,27542.3358,165.9588,0.9914,0.0093,0.0067
3,142.6697,24952.7629,157.9644,0.9856,0.0102,0.0090
4,147.5985,23568.2921,153.5197,0.9901,0.0102,0.0096
5,43.7291,3565.0448,59.7080,0.9990,0.0036,0.0026
6,2371.3831,30906731.4450,5559.3823,-5.6995,0.2175,0.1237
7,102.8265,15240.1331,123.4509,0.9955,0.0081,0.0067
8,138.3121,23851.3025,154.4387,0.9906,0.0106,0.0092
9,135.4981,22510.8933,150.0363,0.9931,0.0092,0.0084


In [39]:
stack_2 = stack_models([huber,dt,lightgbm,ridge], restack = False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,131.3067,19633.8496,140.1208,0.9954,0.0095,0.0087
1,95.5540,12324.8322,111.0173,0.9967,0.0072,0.0061
2,140.7403,35320.9789,187.9388,0.9890,0.0111,0.0084
3,156.1818,33629.8965,183.3846,0.9807,0.0118,0.0099
4,99.8210,13937.3104,118.0564,0.9941,0.0083,0.0067
5,82.4016,9881.9505,99.4080,0.9973,0.0060,0.0050
6,989.2960,5058684.7966,2249.1520,-0.0965,0.1023,0.0518
7,79.1019,9291.0902,96.3903,0.9973,0.0065,0.0052
8,126.5755,23378.5278,152.9004,0.9908,0.0108,0.0085
9,180.0270,34477.2229,185.6804,0.9895,0.0113,0.0111


In [40]:
predict_model(stack_1)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Stacking Regressor,136.4360,41434.5014,203.5547,0.9854,0.0118,0.0083


,gnp,gdpdef,gdp,date_month_4,date_weekday_1,date_is_month_end_0,date_is_month_start_1,month_month_1,month_month_10,month_weekday_3,month_weekday_5,month_is_month_end_0,month_is_month_start_1,quarter_month_7,quarter_weekday_0,quarter_weekday_6,quarter_is_month_end_0,quarter_is_month_start_1,Target,Label
0,18565.144531,104.978996,18332.078125,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,17462.578125,17392.952668
1,20930.650391,110.597000,20659.101562,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,18679.599609,18657.614172
2,14551.583008,95.017998,14430.901367,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,15187.474609,15370.739545
3,10918.383789,80.782997,10887.459961,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,13477.356445,13610.940103
4,18520.402344,104.988998,18306.960938,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,17437.080078,17304.147779
5,17691.164062,103.525002,17462.703125,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,16868.109375,16898.020483
6,14875.943359,93.327003,14715.057617,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,15767.146484,15549.105189
7,19649.441406,112.859001,19477.443359,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,17258.205078,18005.692499
8,17381.812500,102.942001,17144.281250,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,16654.246094,16613.659832
9,13906.646484,90.627998,13870.188477,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,15304.516602,15236.165357


In [41]:
final_stack_1 = finalize_model(stack_1)

In [44]:
unseen_predictions = predict_model(final_stack_1, data=df, round=0)
unseen_predictions.head(6)

,date,month,quarter,Target,gnp,gdpdef,gdp,Label
0,1/1/2000,2000-01,2000Q1,12935.252,10035.263,77.325,10002.179,13180.813641
1,4/1/2000,2000-04,2000Q2,13170.749,10283.391,77.807,10247.720,13324.991171
2,7/1/2000,2000-07,2000Q3,13183.890,10348.645,78.263,10318.165,13372.589674
3,10/1/2000,2000-10,2000Q4,13262.250,10490.430,78.688,10435.744,13441.323044
4,1/1/2001,2001-01,2001Q1,13219.251,10510.297,79.204,10470.231,13374.038206
5,4/1/2001,2001-04,2001Q2,13301.394,10647.427,79.683,10599.000,13495.170767
